## 문자열 데이터로 RNN 학습하기
-문자열 데이터를 벡터로 표현하기 위해 원핫인코딩(One-Hot Encoding)을 사용한다.  
-원핫인코딩으로 표현한 데이터를 가지고 RNN을 학습한다.

In [3]:
import torch
import torch.optim as optim
import numpy as np
import torch.nn.functional as F

## [1] Data Preprocessing
- 단어 'tomato'를 철자 단위에서 학습을 시켜보자

In [6]:
string = 'tomato'
string = list(string)
string

['t', 'o', 'm', 'a', 't', 'o']

In [7]:
set_string = set(string)           # 중복을 허용하지 않는 데이터 타입 set
set_string               # tomato 단어는 t, o, m, a로 구성되어 있음

{'a', 'm', 'o', 't'}

In [10]:
set_string = ['t', 'o', 'm', 'a']

- str데이터를 원핫인코딩으로 표현한다

In [11]:
# input data는 맨마지막 철자를 제외하고 입력한다

encoding_X = [[[ 1, 0, 0, 0 ],   # t --> 0
               [ 0, 1, 0, 0 ],   # o --> 1
               [ 0, 0, 1, 0 ],   # m --> 2
               [ 0, 0, 0, 1 ],   # a --> 3
               [ 1, 0, 0, 0 ]]]  # t --> 0


target = [[ 1, 2, 3, 0, 1 ]] # 처음 시작하는 철자를 제외하고 입력

In [12]:
X = torch.FloatTensor( encoding_X )
Y = torch.LongTensor( target )
X = X.reshape(5, 1, 4)
X.shape

torch.Size([5, 1, 4])

## [2] Model 정의

In [13]:
# hyperparameter 
input_size = 4  #원핫인코딩의 Dimension
hidden_size = 4
epoch = 20
learning_rate = 0.1

In [14]:
#모델 선언
model =  torch.nn.RNN(input_size, hidden_size, batch_first=True)     # batch_first --> (Batch, Seq, input_dim)


In [15]:
# loss function, optimizer 선언
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), learning_rate)


## [3] Training

In [16]:
#training
for i in range(epoch+1):
    optimizer.zero_grad()
    outputs, status  = model(X)
    
    outputs = outputs.reshape(-1, input_size)
    Y = Y.reshape(-1)
    
    loss = criterion(outputs, Y)
    loss.backward()
    optimizer.step()

    result = outputs.detach().numpy().argmax(axis=1)
    result = result.reshape(-1)
    result_string = ''.join([set_string[s] for s in result])

    print(f'epoch:{i} loss:{loss:.2f} --> 예측한 문자:{result_string}, 예측값:{result}')

epoch:0 loss:1.45 --> 예측한 문자:ttttt, 예측값:[0 0 0 0 0]
epoch:1 loss:1.26 --> 예측한 문자:ottto, 예측값:[1 0 0 0 1]
epoch:2 loss:1.12 --> 예측한 문자:otaoo, 예측값:[1 0 3 1 1]
epoch:3 loss:1.03 --> 예측한 문자:ooaoo, 예측값:[1 1 3 1 1]
epoch:4 loss:0.96 --> 예측한 문자:ooaoo, 예측값:[1 1 3 1 1]
epoch:5 loss:0.89 --> 예측한 문자:ooaoo, 예측값:[1 1 3 1 1]
epoch:6 loss:0.83 --> 예측한 문자:ooaoo, 예측값:[1 1 3 1 1]
epoch:7 loss:0.75 --> 예측한 문자:omato, 예측값:[1 2 3 0 1]
epoch:8 loss:0.68 --> 예측한 문자:omato, 예측값:[1 2 3 0 1]
epoch:9 loss:0.61 --> 예측한 문자:omato, 예측값:[1 2 3 0 1]
epoch:10 loss:0.56 --> 예측한 문자:omato, 예측값:[1 2 3 0 1]
epoch:11 loss:0.52 --> 예측한 문자:omato, 예측값:[1 2 3 0 1]
epoch:12 loss:0.49 --> 예측한 문자:omato, 예측값:[1 2 3 0 1]
epoch:13 loss:0.47 --> 예측한 문자:omato, 예측값:[1 2 3 0 1]
epoch:14 loss:0.46 --> 예측한 문자:omato, 예측값:[1 2 3 0 1]
epoch:15 loss:0.45 --> 예측한 문자:omato, 예측값:[1 2 3 0 1]
epoch:16 loss:0.43 --> 예측한 문자:omato, 예측값:[1 2 3 0 1]
epoch:17 loss:0.42 --> 예측한 문자:omato, 예측값:[1 2 3 0 1]
epoch:18 loss:0.41 --> 예측한 문자:omato, 예측값:[1 2 3 0 1]
epo

## [4] Test
-임의의 원핫인코딩 텐서를 입력해서 결과를 확인해본다

In [30]:
test_x = torch.FloatTensor([[1, 0, 0, 0],
                            [0, 1, 0, 0]]) # t o m a 의 원핫인코딩값

test_x = test_x.reshape(2, 1, 4)
test_x.shape

torch.Size([2, 1, 4])

In [31]:
with torch.no_grad():
    test_output, status = model(test_x)
    
    result = test_output.detach().numpy().argmax(axis=2)
    result = result.reshape(-1)
    result_string = ''.join([set_string[s] for s in result])
    print(f'예측한 문자: {result_string}, 예측값:{result}')

예측한 문자: om, 예측값:[1 2]
